In [1]:
import numpy as np
import pandas as pd
from google.colab import files
from utils import *

Carga de datos

In [2]:
path_A = "A - Computer interaction features (Ulog - All Features per minute)-Sheet_1.csv"
path_B = "B - Facial expressions features (FaceReaderAllData_final (NaN is 999))-sheet_1.csv"
path_C = "C - Body posture features (Kinect - final (annotated and selected))-sheet_2.csv"
data_A = cargar_datos_csv(path_A, separador=",", decimal=".")
data_B = cargar_datos_csv(path_B, separador=";", decimal=",")
data_C = cargar_datos_csv(path_C, separador=",", decimal=".")

Datos cargados correctamente: A - Computer interaction features (Ulog - All Features per minute)-Sheet_1.csv
Datos cargados correctamente: B - Facial expressions features (FaceReaderAllData_final (NaN is 999))-sheet_1.csv
Datos cargados correctamente: C - Body posture features (Kinect - final (annotated and selected))-sheet_2.csv


In [3]:

data_A = eliminar_columnas(data_A,["Unnamed: 20", "Unnamed: 21"])

Se eliminaron las columnas: ['Unnamed: 20', 'Unnamed: 21']


In [4]:
data_B = eliminar_columnas(data_B,["Unnamed: 45", "Unnamed: 46"])

Se eliminaron las columnas: ['Unnamed: 45', 'Unnamed: 46']


In [5]:
data_C = eliminar_columnas(data_C,["Added"])

Se eliminaron las columnas: ['Added']


Posibles diferencias entre data set por columnas para mergeado, Condition, PP, Block y timestamp

In [6]:
# Verificar si los timestamps son iguales en los tres datasets
timestamps_iguales = data_A['timestamp'].equals(data_B['timestamp']) and data_A['timestamp'].equals(data_C['timestamp'])

if timestamps_iguales:
    print("Los timestamps son idénticos en los tres datasets.")
else:
    print("Los timestamps tienen diferencias entre los datasets.")

Los timestamps tienen diferencias entre los datasets.


Al analizar las fechas nos damos cuenta que en el data set C hay errores de anotación de fechas, por ejemplo 31/09/2012. Otro error común cometido en ese dataset es poner 08/08/2021 que debería ser 08/09/2012 puesto que el experimento parte en septiembre. Formateamos y corregimos fechas.

In [8]:
formateo_fechas(data_A)
formateo_fechas(data_B)
formateo_fechas(data_C)

,PP,Blok,Condition,timestamp,avgDepth(avg),leftShoulderAngle(avg),rightShoulderAngle(avg),leanAngle(avg),HipCenter_Spine-Spine_ShoulderCenter(avg),Spine_ShoulderCenter-ShoulderCenter_Head(avg),...,WristLeft_HandLeft-PlaneYZAxisZ(stdv),ShoulderRight_ElbowRight-PlaneZXAxisX(stdv),ShoulderRight_ElbowRight-PlaneXYAxisY(stdv),ShoulderRight_ElbowRight-PlaneYZAxisZ(stdv),ElbowRight_WristRight-PlaneZXAxisX(stdv),ElbowRight_WristRight-PlaneXYAxisY(stdv),ElbowRight_WristRight-PlaneYZAxisZ(stdv),WristRight_HandRight-PlaneZXAxisX(stdv),WristRight_HandRight-PlaneXYAxisY(stdv),WristRight_HandRight-KinectZAxis(stdv)
0,PP1,1,R,2012-08-18 13:16:00,2102.597393,-116.055931,115.017758,92.340895,176.389544,155.409463,...,166.872086,22.946337,5.605230,13.237048,30.932939,102.460262,8.444481,54.436879,75.886750,155.734343
1,PP1,1,R,2012-08-18 13:17:00,2099.725525,-116.301605,115.986636,92.083385,178.236949,155.185113,...,58.237714,7.194860,2.468881,3.886001,2.000359,3.431337,1.160298,4.579341,2.868843,171.486911
2,PP1,1,R,2012-08-18 13:18:00,2102.365778,-115.963089,114.073054,92.381690,170.876998,155.363873,...,64.548824,14.726365,2.953021,9.185623,13.379140,48.162012,30.508877,23.013755,30.629061,149.748809
3,PP1,1,R,2012-08-18 13:19:00,2104.116968,-115.629630,113.972465,92.428905,168.248852,156.627814,...,38.209895,2.542686,1.564318,2.453674,4.796273,8.223865,1.114906,3.854074,4.048611,173.149430
4,PP1,1,R,2012-08-18 13:20:00,2105.284007,-116.359699,111.538437,92.461537,166.050980,152.687506,...,80.171422,9.905241,20.342488,5.552234,7.198096,22.942488,46.386278,17.514534,29.702675,165.368301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134,PP25,3,T,2012-10-07 16:15:00,2047.087447,-115.219804,116.330627,104.529078,176.843390,160.731168,...,103.650684,36.355940,5.323538,17.908446,56.076888,124.607529,15.482486,87.838259,158.284130,38.905423
3135,PP25,3,T,2012-10-07 16:16:00,2048.832532,-115.124481,115.231220,102.381525,176.970969,154.387065,...,26.334438,14.714479,1.848907,3.908359,34.878709,159.984897,5.233569,63.512098,164.607803,27.895148
3136,PP25,3,T,2012-10-07 16:17:00,2048.863806,-115.138635,113.786573,103.612981,176.990190,156.480959,...,15.970939,22.742133,3.048023,6.468803,53.988308,169.548392,10.757358,60.488127,142.557094,21.761699
3137,PP25,3,T,2012-10-07 16:18:00,2049.784363,-114.085126,109.726265,108.599482,176.763443,159.023415,...,38.797002,28.595668,27.981837,8.424373,43.183524,55.599939,14.620391,72.590531,163.087296,28.944050


Una persona no tiene el mismo estado de ánimo en la mañana que en la tarde, por lo que se aplica un clasificador Mañana, Tarde o Noche a las fechas.

In [9]:
#Clasificamos los momentos del día
data_A = clasificar_momento_dia(data_A)
data_B = clasificar_momento_dia(data_B)
data_C = clasificar_momento_dia(data_C)

In [10]:
#Comprobamos consistencia en los momentos del día por posición absoluta en el data set
orden_momento_dia_A = data_A["momento_dia"].tolist()
orden_momento_dia_B = data_B["momento_dia"].tolist()
orden_momento_dia_C = data_C["momento_dia"].tolist()

orden_momento_dia_A == orden_momento_dia_B == orden_momento_dia_C


True

In [11]:
#Comprobamos consistencia en los momentos del día por conteo en los data sets
conteo_momento_dia_A = data_A['momento_dia'].value_counts()
conteo_momento_dia_B = data_B['momento_dia'].value_counts()
conteo_momento_dia_C = data_C['momento_dia'].value_counts()


print("Conteo de momentos del día en Data_A:")
print(conteo_momento_dia_A)
print("\nConteo de momentos del día en Data_B:")
print(conteo_momento_dia_B)
print("\nConteo de momentos del día en Data_C:")
print(conteo_momento_dia_C)

Conteo de momentos del día en Data_A:
momento_dia
tarde     2363
mañana     774
noche        2
Name: count, dtype: int64

Conteo de momentos del día en Data_B:
momento_dia
tarde     2363
mañana     774
noche        2
Name: count, dtype: int64

Conteo de momentos del día en Data_C:
momento_dia
tarde     2363
mañana     774
noche        2
Name: count, dtype: int64


In [12]:
orden_blok_A = data_A["Blok"].tolist()
orden_blok_B = data_B["Blok"].tolist()
orden_blok_C = data_C["Blok"].tolist()

orden_blok_A == orden_blok_B == orden_blok_C

True

In [13]:
orden_pp_A = data_A["PP"].tolist()
orden_pp_B = data_B["PP"].tolist()
orden_pp_C = data_C["PP"].tolist()

orden_pp_A == orden_pp_B == orden_pp_C

True

In [14]:
orden_condition_A = data_A["Condition"].tolist()
orden_condition_B = data_B["Condition"].tolist()
orden_condition_C = data_C["Condition"].tolist()

orden_condition_A == orden_condition_B == orden_condition_C

True

In [15]:
orden_ts_A = data_A["timestamp"].tolist()
orden_ts_B = data_B["timestamp"].tolist()
orden_ts_C = data_C["timestamp"].tolist()

orden_ts_A == orden_ts_B == orden_ts_C

False

In [16]:
# Columnas a comparar
cols = ['timestamp', 'PP', 'Blok', 'Condition', 'momento_dia']

# Encontrar inconsistencias
tabla_inconsistencias = inconsistencias_cat_cols(data_A, data_B, data_C, cols)

# Agrupar por día y mostrar inconsistencias
tabla_agrupada = tabla_inconsistencias.groupby(['Fecha', 'Columna']).agg({
    'Valor_A': list,
    'Valor_B': list,
    'Valor_C': list
}).reset_index()

# Mostrar la tabla de inconsistencias agrupada
tabla_agrupada

,Fecha,Columna,Valor_A,Valor_B,Valor_C
0,2012-09-18,timestamp,"[2012-09-18 13:16:00, 2012-09-18 13:17:00, 201...","[2012-09-18 13:16:00, 2012-09-18 13:17:00, 201...","[2012-08-18 13:16:00, 2012-08-18 13:17:00, 201..."
1,2012-09-19,timestamp,"[2012-09-19 13:12:00, 2012-09-19 13:13:00, 201...","[2012-09-19 13:12:00, 2012-09-19 13:13:00, 201...","[2012-08-19 13:12:00, 2012-08-19 13:13:00, 201..."
2,2012-09-24,timestamp,"[2012-09-24 13:24:00, 2012-09-24 13:25:00, 201...","[2012-09-24 13:24:00, 2012-09-24 13:25:00, 201...","[2012-08-24 13:24:00, 2012-08-24 13:25:00, 201..."
3,2012-09-25,timestamp,"[2012-09-25 13:16:00, 2012-09-25 13:17:00, 201...","[2012-09-25 13:16:00, 2012-09-25 13:17:00, 201...","[2012-08-25 13:16:00, 2012-08-25 13:17:00, 201..."
4,2012-09-26,timestamp,"[2012-09-26 09:27:00, 2012-09-26 09:28:00, 201...","[2012-09-26 09:27:00, 2012-09-26 09:28:00, 201...","[2012-08-26 09:27:00, 2012-08-26 09:28:00, 201..."
5,2012-10-01,timestamp,"[2012-10-01 14:14:00, 2012-10-01 14:15:00, 201...","[2012-10-01 14:14:00, 2012-10-01 14:15:00, 201...","[2012-09-01 14:14:00, 2012-09-01 14:15:00, 201..."
6,2012-10-02,timestamp,"[2012-10-02 13:23:00, 2012-10-02 13:24:00, 201...","[2012-10-02 13:23:00, 2012-10-02 13:24:00, 201...","[2012-09-02 13:23:00, 2012-09-02 13:24:00, 201..."
7,2012-10-04,timestamp,"[2012-10-04 09:34:00, 2012-10-04 09:35:00, 201...","[2012-10-04 09:34:00, 2012-10-04 09:35:00, 201...","[2012-09-04 09:34:00, 2012-09-04 09:35:00, 201..."
8,2012-10-08,timestamp,"[2012-10-08 13:12:00, 2012-10-08 13:13:00, 201...","[2012-10-08 13:12:00, 2012-10-08 13:13:00, 201...","[2012-09-08 13:12:00, 2012-09-08 13:13:00, 201..."
9,2012-10-09,timestamp,"[2012-10-09 09:23:00, 2012-10-09 09:24:00, 201...","[2012-10-09 09:23:00, 2012-10-09 09:24:00, 201...","[2012-09-09 09:23:00, 2012-09-09 09:24:00, 201..."


Una vez vista las inconsistencias ya podemos podemos concatenar los tres datasets para obtener uno único con todas las features. Se puede observar que el subset C tiene fallos de anotación corriendo un mes hacia atras del mes real del experimento. A esta conclusión se llega porque los experimento se hacian en un periodo de tres horas por particpante en un día en concreto, por lo que es un fallo de anotación de la fecha, por ejemplo, 2012-09-18 es la fecha correcta y en C aparece como 2012-08-18, sin embargo hora y minutos aparecen exactamente igules en todos, de ahí que la clasificación por mañana, tarde y noche siga el mismo orden en las tres tablas.

In [18]:
data_A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3139 entries, 0 to 3138
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   PP                3139 non-null   object        
 1   Blok              3139 non-null   int64         
 2   Condition         3139 non-null   object        
 3   timestamp         3139 non-null   datetime64[ns]
 4   SnMouseAct        3139 non-null   int64         
 5   SnLeftClicked     3139 non-null   int64         
 6   SnRightClicked    3139 non-null   int64         
 7   SnDoubleClicked   3139 non-null   int64         
 8   SnWheel           3139 non-null   int64         
 9   SnDragged         3139 non-null   int64         
 10  SnMouseDistance   3139 non-null   object        
 11  SnKeyStrokes      3139 non-null   int64         
 12  SnChars           3139 non-null   int64         
 13  SnSpecialKeys     3139 non-null   int64         
 14  SnDirectionKeys   3139 n

In [19]:
#Al ver que PP, Blok Condition, timestamp y momento día siguen el mismo orden en todos los dataset, hay que borrarlo en todos menos uno.
data_C = data_C.drop(columns=["PP", "Blok", "Condition", "momento_dia", "timestamp"])
data_B = data_B.drop(columns=["PP", "Blok", "Condition", "momento_dia", "timestamp"])

In [20]:
#Movemos en Data momento del día a cuarta posición
data_A = data_A.reindex(columns = data_A.columns[:4].tolist() +
                         ["momento_dia"] +
                         [col for col in data_A.columns if col not in data_A.columns[:4].tolist() + ["momento_dia"]])

In [21]:
data_A["SnMouseDistance"] = data_A["SnMouseDistance"].replace("#VALUE!", np.nan)
data_A["SnMouseDistance"] = pd.to_numeric(data_A["SnMouseDistance"], errors='coerce')
data_A[["PP", "Blok", "Condition", "momento_dia"]] = data_A[["PP", "Blok", "Condition", "momento_dia"]].astype("string")

In [22]:
data_A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3139 entries, 0 to 3138
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   PP                3139 non-null   string        
 1   Blok              3139 non-null   string        
 2   Condition         3139 non-null   string        
 3   timestamp         3139 non-null   datetime64[ns]
 4   momento_dia       3139 non-null   string        
 5   SnMouseAct        3139 non-null   int64         
 6   SnLeftClicked     3139 non-null   int64         
 7   SnRightClicked    3139 non-null   int64         
 8   SnDoubleClicked   3139 non-null   int64         
 9   SnWheel           3139 non-null   int64         
 10  SnDragged         3139 non-null   int64         
 11  SnMouseDistance   3137 non-null   float64       
 12  SnKeyStrokes      3139 non-null   int64         
 13  SnChars           3139 non-null   int64         
 14  SnSpecialKeys     3139 n

In [23]:
data_B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3139 entries, 0 to 3138
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Squality                  3139 non-null   float64
 1   Sneutral                  3139 non-null   float64
 2   Shappy                    3139 non-null   float64
 3   Ssad                      3139 non-null   float64
 4   Sangry                    3139 non-null   float64
 5   Ssurprised                3139 non-null   float64
 6   Sscared                   3139 non-null   float64
 7   Sdisgusted                3139 non-null   float64
 8   Svalence                  3139 non-null   float64
 9   SyHeadOrientation         3139 non-null   float64
 10  SxHeadOrientation         3139 non-null   float64
 11  SzHeadOrientation         3139 non-null   float64
 12  SmouthOpen                3139 non-null   float64
 13  SleftEyeClosed            3139 non-null   float64
 14  SrightEy

In [24]:
data_C.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3139 entries, 0 to 3138
Data columns (total 94 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   avgDepth(avg)                                                2741 non-null   float64
 1   leftShoulderAngle(avg)                                       2741 non-null   float64
 2   rightShoulderAngle(avg)                                      2738 non-null   float64
 3   leanAngle(avg)                                               2738 non-null   float64
 4   HipCenter_Spine-Spine_ShoulderCenter(avg)                    2738 non-null   float64
 5   Spine_ShoulderCenter-ShoulderCenter_Head(avg)                2738 non-null   float64
 6   Spine_ShoulderCenter-ShoulderCenter_ShoulderLeft(avg)        2738 non-null   float64
 7   Spine_ShoulderCenter-ShoulderCenter_ShoulderRight(avg)       2738 non-null   f

In [25]:
full_dataset = pd.concat([data_A, data_B, data_C], axis=1)

In [26]:
full_dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3139 entries, 0 to 3138
Data columns (total 156 columns):
 #    Column                                                       Dtype         
---   ------                                                       -----         
 0    PP                                                           string        
 1    Blok                                                         string        
 2    Condition                                                    string        
 3    timestamp                                                    datetime64[ns]
 4    momento_dia                                                  string        
 5    SnMouseAct                                                   int64         
 6    SnLeftClicked                                                int64         
 7    SnRightClicked                                               int64         
 8    SnDoubleClicked                                              int64

In [27]:
#Convertir los datos numericos a float 64 para tener más precision
for col in full_dataset.select_dtypes(include=['int64']).columns:
    full_dataset[col] = full_dataset[col].astype('float64')

In [28]:
#Reemplazar los 999 a nan
full_dataset = full_dataset.replace(999.0, np.nan)

In [29]:
full_dataset.to_csv('full_dataset.csv', index=False)
files.download('full_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
full_dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3139 entries, 0 to 3138
Data columns (total 156 columns):
 #    Column                                                       Dtype         
---   ------                                                       -----         
 0    PP                                                           string        
 1    Blok                                                         string        
 2    Condition                                                    string        
 3    timestamp                                                    datetime64[ns]
 4    momento_dia                                                  string        
 5    SnMouseAct                                                   float64       
 6    SnLeftClicked                                                float64       
 7    SnRightClicked                                               float64       
 8    SnDoubleClicked                                              float

In [31]:
full_dataset.describe().T

,count,mean,min,25%,50%,75%,max,std
timestamp,3139,2012-10-12 15:13:59.694170112,2012-09-18 13:16:00,2012-10-02 13:53:30,2012-10-10 10:58:00,2012-10-19 15:55:30,2012-11-07 16:19:00,NaN
SnMouseAct,3108.0,13.512548,0.0,0.0,9.0,22.0,84.0,14.584587
SnLeftClicked,3108.0,6.049228,0.0,0.0,4.0,10.0,38.0,6.529561
SnRightClicked,3108.0,0.202703,0.0,0.0,0.0,0.0,10.0,0.66459
SnDoubleClicked,3108.0,0.369691,0.0,0.0,0.0,0.0,25.0,1.264023
SnWheel,3108.0,3.410553,0.0,0.0,1.0,5.0,80.0,6.123242
SnDragged,3108.0,1.723938,0.0,0.0,1.0,2.0,25.0,2.780984
SnMouseDistance,3106.0,9067.431827,0.0,0.0,5448.370276,13607.554362,81098.584859,11497.019906
SnKeyStrokes,3108.0,72.877413,0.0,3.0,53.0,120.0,382.0,74.482327
SnChars,3108.0,45.774453,0.0,0.0,29.0,75.0,298.0,50.93847


In [32]:
#Guardar y descargar full_dataset para trabajar en el posteriormente (y tener una copia de seguridad)
full_dataset.to_csv('full_dataset.csv', index=False)
files.download('full_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Breve descripción y clasificación de las características de full_dataset


1. **Registros de Computadora (Interacciones con el Ratón y Teclado)**
Estas características reflejan la actividad del usuario en el ordenador.

    **1.1. Eventos del ratón**
    * SnMouseAct → Número total de eventos del ratón.
    * SnLeftClicked → Número de clics izquierdos.
    * SnRightClicked → Número de clics derechos.
    * SnDoubleClicked → Número de doble clics.
    * SnWheel → Número de veces que se usó la rueda del ratón.
    * SnDragged → Número de veces que se arrastró un elemento.
    * SnMouseDistance → Distancia recorrida por el cursor del ratón.

    **1.2. Eventos del teclado**
    * SnKeyStrokes → Número total de pulsaciones de teclas.
    * SnChars → Número de caracteres ingresados.
    * SnSpecialKeys → Número de teclas especiales presionadas (Ej. Shift, Ctrl, Alt).
    * SnDirectionKeys → Número de veces que se usaron las teclas direccionales.
    * SnErrorKeys → Número de veces que se presionaron teclas de corrección (Ej. Backspace).
    * SnShortcutKeys → Número de atajos de teclado usados.
    * SnSpaces → Número de veces que se presionó la barra espaciadora.
    **1.3. Cambios en la interfaz**
    * SnAppChange → Número de cambios entre aplicaciones activas.
    * SnTabfocusChange → Número de cambios de enfoque entre pestañas o ventanas.

2. **Expresiones Faciales (Grabaciones de Cámara)**
Medidas relacionadas con emociones, orientación y movimientos faciales.

    **2.1. Emociones detectadas**
    * Sneutral → Nivel de neutralidad.
    * Shappy → Nivel de felicidad.
    * Ssad → Nivel de tristeza.
    * Sangry → Nivel de enojo.
    * Ssurprised → Nivel de sorpresa.
    * Sscared → Nivel de miedo.
    * Sdisgusted → Nivel de asco.
    * Svalence → Valoración del estado emocional (positivo o negativo).

    **2.2. Orientación de la cabeza**
    * SyHeadOrientation → Orientación en el eje Y.
    * SxHeadOrientation → Orientación en el eje X.
    * SzHeadOrientation → Orientación en el eje Z.
    **2.3. Movimiento de la boca y ojos**
    * SmouthOpen → Nivel de apertura de la boca.
    * SleftEyeClosed → Nivel de cierre del ojo izquierdo.
    * SrightEyeClosed → Nivel de cierre del ojo derecho.
    ** 2.4. Movimiento de las cejas**
    * SleftEyebrowLowered → Descenso de la ceja izquierda.
    * SleftEyebrowRaised → Elevación de la ceja izquierda.
    * SrightEyebrowLowered → Descenso de la ceja derecha.
    * SrightEyebrowRaised → Elevación de la ceja derecha.
    **2.5. Dirección de la mirada**
    * SgazeDirectionForward → Mirada al frente.
    * SgazeDirectionLeft → Mirada a la izquierda.
    * SgazeDirectionRight → Mirada a la derecha.
    **2.6. Unidades de Acción Facial (AUs)**
    Movimientos faciales basados en el sistema FACS:

    * SAu01_InnerBrowRaiser → Elevación interna de cejas.
    * SAu02_OuterBrowRaiser → Elevación externa de cejas.
    * SAu04_BrowLowerer → Descenso de cejas.
    * SAu05_UpperLidRaiser → Elevación del párpado superior.
    * SAu06_CheekRaiser → Elevación de mejillas.
    * SAu07_LidTightener → Tensión en los párpados.
    * SAu09_NoseWrinkler → Arrugamiento de la nariz.
    * SAu10_UpperLipRaiser → Elevación del labio superior.
    * SAu12_LipCornerPuller → Elevación de las comisuras de los labios.
    * SAu14_Dimpler → Formación de hoyuelos.
    * SAu15_LipCornerDepressor → Descenso de las comisuras de los labios.
    * SAu17_ChinRaiser → Elevación del mentón.
    * SAu20_LipStretcher → Estiramiento de labios.
    * SAu23_LipTightener → Tensión en los labios.
    * SAu24_LipPressor → Presión de labios.
    * SAu25_LipsPart → Separación de labios.
    * SAu26_JawDrop → Descenso de la mandíbula.
    * SAu27_MouthStretch → Estiramiento de la boca.
    * SAu43_EyesClosed → Cierre de los ojos.

3. **Posturas Corporales (Sensor Kinect 3D)**
Miden la posición, inclinación y movimiento del cuerpo en 3D.

    3.1. **Ángulos y profundidad**

        * avgDepth(avg), avgDepth(stdv) → Profundidad media y su desviación estándar.
        * leftShoulderAngle(avg), leftShoulderAngle(stdv) → Ángulo del hombro izquierdo.
        * rightShoulderAngle(avg), rightShoulderAngle(stdv) → Ángulo del hombro derecho.
        * leanAngle(avg), leanAngle(stdv) → Ángulo de inclinación del torso.

    3.2. **Segmentos corporales (promedios y desviaciones estándar)**
    Registros de articulaciones y relaciones entre segmentos corporales, como:

        * HipCenter_Spine-Spine_ShoulderCenter (caderas a hombros).
        * ShoulderCenter_ShoulderLeft-ShoulderLeft_ElbowLeft (hombro a codo).
        * ElbowLeft_WristLeft-WristLeft_HandLeft (codo a mano).
        * ShoulderRight_ElbowRight-ElbowRight_WristRight (brazo derecho).
        * Spine_ShoulderCenter-PlaneXYAxisY (proyección en ejes espaciales).
        
4. **Variables Contextuales**
Definen las condiciones en las que se registraron los datos.

    * Blok_2, Blok_3 → Bloques de experimento.
    * Condition_N, Condition_R, Condition_T → Tipos de condiciones experimentales.
    * momento_dia_noche, momento_dia_tarde → Momento del día en que se tomaron los datos.

